# 0.2.0 Feature Engineering

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

import utils.paths as path
from utils.paths2 import direcciones

## 0.2.2 Derive age difference

In [4]:
org_final = pd.read_csv(path.data_raw_dir("org_final.csv"), sep=',')
org_final['date_of_joining'] = pd.to_datetime(org_final['date_of_joining'])
org_final['cutoff_date'] = pd.to_datetime(org_final['cutoff_date'])
org_final['last_working_date'] = pd.to_datetime(org_final['last_working_date'])
print(org_final.info())
org_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   emp_id                  1954 non-null   object        
 1   status                  1954 non-null   object        
 2   location                1954 non-null   object        
 3   level                   1954 non-null   object        
 4   gender                  1954 non-null   object        
 5   emp_age                 1954 non-null   float64       
 6   rating                  1954 non-null   object        
 7   mgr_rating              1954 non-null   object        
 8   mgr_reportees           1954 non-null   int64         
 9   mgr_age                 1954 non-null   float64       
 10  mgr_tenure              1954 non-null   float64       
 11  compensation            1954 non-null   int64         
 12  percent_hike            1954 non-null   int64   

,emp_id,status,location,level,gender,emp_age,rating,mgr_rating,mgr_reportees,mgr_age,mgr_tenure,compensation,percent_hike,hiring_score,hiring_source,no_companies_worked,distance_from_home,total_dependents,marital_status,education,promotion_last_2_years,no_leaves_taken,total_experience,monthly_overtime_hrs,date_of_joining,last_working_date,department,mgr_id,cutoff_date,turnover,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction,age_diff,job_hop_index,tenure,median_compensation,compa_ratio,compa_level
0,E10012,Active,New York,Analyst,Female,25.09,Above Average,Acceptable,9,44.07,3.17,64320,10,70,Consultant,1,14,2,Single,Bachelors,No,2,6.86,1,2011-06-03,NaT,Customer Operations,E9335,2014-12-31,0,0.730,0.73,0.73,0.75,18.98,6.860000,3.821918,51840,1.240741,Above
1,E10025,Active,Chicago,Analyst,Female,25.98,Acceptable,Excellent,4,35.99,7.92,48204,8,70,Job Fairs,9,21,2,Single,Bachelors,No,10,4.88,5,2009-09-23,NaT,Customer Operations,E6655,2014-12-31,0,0.581,0.72,0.84,0.85,10.01,0.542222,5.271233,51840,0.929861,Below
2,E10027,Active,Orlando,Specialist,Female,33.40,Acceptable,Above Average,6,35.78,4.38,85812,11,77,Consultant,3,15,5,Single,Bachelors,Yes,18,8.55,3,2005-02-11,NaT,Customer Operations,E13942,2014-12-31,0,0.770,0.85,0.80,0.87,2.38,2.850000,9.161644,83496,1.027738,Above
3,E10048,Active,Chicago,Specialist,Male,24.55,Acceptable,Acceptable,10,26.70,2.87,49536,8,71,Job Boards,5,9,3,Single,Bachelors,Yes,19,4.76,8,2011-05-20,NaT,Customer Operations,E7063,2014-12-31,0,0.240,0.42,0.33,0.85,2.15,0.952000,3.616438,83496,0.593274,Below
4,E10060,Active,Orlando,Analyst,Male,31.23,Acceptable,Acceptable,11,34.28,12.95,75576,12,70,Job Fairs,1,25,4,Single,Bachelors,No,25,8.06,1,2011-12-21,NaT,Customer Operations,E5663,2014-12-31,0,0.710,0.78,0.67,0.80,3.05,8.060000,3.027397,51840,1.457870,Above


In [5]:
emp_age_diff = org_final.copy()
emp_age_diff['age_diff'] = org_final['mgr_age'] - org_final['emp_age']

In [6]:
fig = px.box(emp_age_diff, x='status', y="age_diff")
fig.show()

There is a lot of overlap between the distributions for inactive and active employees. The median age difference for inactive employees is slightly higher than for active employees, but we don't know if this is meaningful or statistically significant.

## 0.2.3 Derive job hop index

**incluir imagen**

In [7]:
emp_jhi = emp_age_diff.copy()
emp_jhi['job_hop_index'] = emp_jhi['total_experience'] / emp_jhi['no_companies_worked']

In [8]:
fig = px.box(emp_jhi, x='status', y="job_hop_index")
fig.show()

Median job hop index for active and inactive employee are similar!

## 0.2.4 Derive employee tenure

In [9]:
emp_tenure = emp_jhi.copy()
emp_tenure['tenure'] = np.where(emp_tenure['status'] == 'Active', (emp_tenure['cutoff_date'] - emp_tenure['date_of_joining']), (emp_tenure['last_working_date'] - emp_tenure['date_of_joining']))
emp_tenure['tenure'] = emp_tenure['tenure'].dt.days / 365.25
emp_tenure.head()

,emp_id,status,location,level,gender,emp_age,rating,mgr_rating,mgr_reportees,mgr_age,mgr_tenure,compensation,percent_hike,hiring_score,hiring_source,no_companies_worked,distance_from_home,total_dependents,marital_status,education,promotion_last_2_years,no_leaves_taken,total_experience,monthly_overtime_hrs,date_of_joining,last_working_date,department,mgr_id,cutoff_date,turnover,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction,age_diff,job_hop_index,tenure,median_compensation,compa_ratio,compa_level
0,E10012,Active,New York,Analyst,Female,25.09,Above Average,Acceptable,9,44.07,3.17,64320,10,70,Consultant,1,14,2,Single,Bachelors,No,2,6.86,1,2011-06-03,NaT,Customer Operations,E9335,2014-12-31,0,0.730,0.73,0.73,0.75,18.98,6.860000,3.578371,51840,1.240741,Above
1,E10025,Active,Chicago,Analyst,Female,25.98,Acceptable,Excellent,4,35.99,7.92,48204,8,70,Job Fairs,9,21,2,Single,Bachelors,No,10,4.88,5,2009-09-23,NaT,Customer Operations,E6655,2014-12-31,0,0.581,0.72,0.84,0.85,10.01,0.542222,5.270363,51840,0.929861,Below
2,E10027,Active,Orlando,Specialist,Female,33.40,Acceptable,Above Average,6,35.78,4.38,85812,11,77,Consultant,3,15,5,Single,Bachelors,Yes,18,8.55,3,2005-02-11,NaT,Customer Operations,E13942,2014-12-31,0,0.770,0.85,0.80,0.87,2.38,2.850000,9.883641,83496,1.027738,Above
3,E10048,Active,Chicago,Specialist,Male,24.55,Acceptable,Acceptable,10,26.70,2.87,49536,8,71,Job Boards,5,9,3,Single,Bachelors,Yes,19,4.76,8,2011-05-20,NaT,Customer Operations,E7063,2014-12-31,0,0.240,0.42,0.33,0.85,2.15,0.952000,3.616701,83496,0.593274,Below
4,E10060,Active,Orlando,Analyst,Male,31.23,Acceptable,Acceptable,11,34.28,12.95,75576,12,70,Job Fairs,1,25,4,Single,Bachelors,No,25,8.06,1,2011-12-21,NaT,Customer Operations,E5663,2014-12-31,0,0.710,0.78,0.67,0.80,3.05,8.060000,3.028063,51840,1.457870,Above


In [10]:
emp_tenure['tenure'].mean()

4.436900175634388

In [11]:
fig = px.box(emp_tenure, x='status', y="tenure")
fig.show()

The median tenure of inactive employees is less than the tenure of active employees.

## 0.2.6 Exploring compensation

**incluir imagen**

In [12]:
fig = px.histogram(emp_tenure, x="compensation")
fig.show()

In [13]:
fig = px.box(emp_tenure, x='level', y="compensation")
fig.show()

In [14]:
fig = px.box(emp_tenure, x='level', y="compensation", color='status')
fig.show()

noticed the variation of compensation within Analyst and Specialist levels. At Analyst level, median compensation of Inactive employees is low when compared to Active employees.

## 0.2.8 Deriving Compa-ratio

**incluir imagen**

In [15]:
emp_compa_ratio = emp_tenure.copy()
median_compensation = emp_compa_ratio.groupby('level').mean().reset_index()
median_compensation['compensation']

0    54874.885287
1    82998.857143
Name: compensation, dtype: float64

In [16]:
emp_compa_ratio['compa_ratio'] = emp_compa_ratio['compensation'].groupby(emp_compa_ratio['level']).apply(lambda x: x/x.mean())
emp_compa_ratio.head()

,emp_id,status,location,level,gender,emp_age,rating,mgr_rating,mgr_reportees,mgr_age,mgr_tenure,compensation,percent_hike,hiring_score,hiring_source,no_companies_worked,distance_from_home,total_dependents,marital_status,education,promotion_last_2_years,no_leaves_taken,total_experience,monthly_overtime_hrs,date_of_joining,last_working_date,department,mgr_id,cutoff_date,turnover,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction,age_diff,job_hop_index,tenure,median_compensation,compa_ratio,compa_level
0,E10012,Active,New York,Analyst,Female,25.09,Above Average,Acceptable,9,44.07,3.17,64320,10,70,Consultant,1,14,2,Single,Bachelors,No,2,6.86,1,2011-06-03,NaT,Customer Operations,E9335,2014-12-31,0,0.730,0.73,0.73,0.75,18.98,6.860000,3.578371,51840,1.172121,Above
1,E10025,Active,Chicago,Analyst,Female,25.98,Acceptable,Excellent,4,35.99,7.92,48204,8,70,Job Fairs,9,21,2,Single,Bachelors,No,10,4.88,5,2009-09-23,NaT,Customer Operations,E6655,2014-12-31,0,0.581,0.72,0.84,0.85,10.01,0.542222,5.270363,51840,0.878435,Below
2,E10027,Active,Orlando,Specialist,Female,33.40,Acceptable,Above Average,6,35.78,4.38,85812,11,77,Consultant,3,15,5,Single,Bachelors,Yes,18,8.55,3,2005-02-11,NaT,Customer Operations,E13942,2014-12-31,0,0.770,0.85,0.80,0.87,2.38,2.850000,9.883641,83496,1.033894,Above
3,E10048,Active,Chicago,Specialist,Male,24.55,Acceptable,Acceptable,10,26.70,2.87,49536,8,71,Job Boards,5,9,3,Single,Bachelors,Yes,19,4.76,8,2011-05-20,NaT,Customer Operations,E7063,2014-12-31,0,0.240,0.42,0.33,0.85,2.15,0.952000,3.616701,83496,0.596827,Below
4,E10060,Active,Orlando,Analyst,Male,31.23,Acceptable,Acceptable,11,34.28,12.95,75576,12,70,Job Fairs,1,25,4,Single,Bachelors,No,25,8.06,1,2011-12-21,NaT,Customer Operations,E5663,2014-12-31,0,0.710,0.78,0.67,0.80,3.05,8.060000,3.028063,51840,1.377242,Above


## 0.2.9 Deriving Compa-level

In [17]:
emp_final = emp_compa_ratio.copy()
emp_final['compa_level'] = np.where(emp_final['compa_ratio'] >= 1, 'Above', 'Below')
emp_final.head()

,emp_id,status,location,level,gender,emp_age,rating,mgr_rating,mgr_reportees,mgr_age,mgr_tenure,compensation,percent_hike,hiring_score,hiring_source,no_companies_worked,distance_from_home,total_dependents,marital_status,education,promotion_last_2_years,no_leaves_taken,total_experience,monthly_overtime_hrs,date_of_joining,last_working_date,department,mgr_id,cutoff_date,turnover,mgr_effectiveness,career_satisfaction,perf_satisfaction,work_satisfaction,age_diff,job_hop_index,tenure,median_compensation,compa_ratio,compa_level
0,E10012,Active,New York,Analyst,Female,25.09,Above Average,Acceptable,9,44.07,3.17,64320,10,70,Consultant,1,14,2,Single,Bachelors,No,2,6.86,1,2011-06-03,NaT,Customer Operations,E9335,2014-12-31,0,0.730,0.73,0.73,0.75,18.98,6.860000,3.578371,51840,1.172121,Above
1,E10025,Active,Chicago,Analyst,Female,25.98,Acceptable,Excellent,4,35.99,7.92,48204,8,70,Job Fairs,9,21,2,Single,Bachelors,No,10,4.88,5,2009-09-23,NaT,Customer Operations,E6655,2014-12-31,0,0.581,0.72,0.84,0.85,10.01,0.542222,5.270363,51840,0.878435,Below
2,E10027,Active,Orlando,Specialist,Female,33.40,Acceptable,Above Average,6,35.78,4.38,85812,11,77,Consultant,3,15,5,Single,Bachelors,Yes,18,8.55,3,2005-02-11,NaT,Customer Operations,E13942,2014-12-31,0,0.770,0.85,0.80,0.87,2.38,2.850000,9.883641,83496,1.033894,Above
3,E10048,Active,Chicago,Specialist,Male,24.55,Acceptable,Acceptable,10,26.70,2.87,49536,8,71,Job Boards,5,9,3,Single,Bachelors,Yes,19,4.76,8,2011-05-20,NaT,Customer Operations,E7063,2014-12-31,0,0.240,0.42,0.33,0.85,2.15,0.952000,3.616701,83496,0.596827,Below
4,E10060,Active,Orlando,Analyst,Male,31.23,Acceptable,Acceptable,11,34.28,12.95,75576,12,70,Job Fairs,1,25,4,Single,Bachelors,No,25,8.06,1,2011-12-21,NaT,Customer Operations,E5663,2014-12-31,0,0.710,0.78,0.67,0.80,3.05,8.060000,3.028063,51840,1.377242,Above


In [18]:
fig = px.histogram(emp_final, x='status' , color='compa_level', barnorm = "percent", histnorm='probability density') #, 
fig.show()

A greater proportion of inactive employees were paid less than median compensation.

## 0.2.10 Calculating Information Value

## 0.2.11 Which variables are important?

**investigar**

In [19]:
print('ok_')

ok_
